# Phần 2: Phân tích Dữ liệu (Data Analysis)

## 2.1. Giới thiệu

Sau khi hoàn thành việc thu thập, khám phá và làm sạch dữ liệu ở Phần 1, chúng em đã có một bộ dữ liệu đáng tin cậy (`covid_cleaned_country_data.csv`). Trong phần này, chúng em sẽ tiến hành phân tích sâu hơn vào bộ dữ liệu này để rút ra những thông tin chi tiết và trả lời các câu hỏi nghiệp vụ đã đặt ra.

Mục tiêu của Phần 2 là sử dụng các thư viện phân tích dữ liệu của Python, chủ yếu là **Pandas**, để thực hiện các phép tính, tổng hợp và so sánh, nhằm mục đích:
*   Phân tích tình hình dịch bệnh trên quy mô toàn cầu và theo từng quốc gia.
*   Khám phá xu hướng lây lan và tác động của đại dịch theo thời gian.
*   Tìm hiểu mối tương quan giữa các yếu tố dịch tễ, kinh tế và xã hội.

Các phân tích trong phần này sẽ là nền tảng vững chắc cho việc trực quan hóa dữ liệu ở Phần 3 và xây dựng mô hình dự đoán ở Phần 5.

## 2.2. Chuẩn bị Môi trường và Dữ liệu

Trước khi bắt đầu, em ta cần tải các thư viện cần thiết và đọc lại bộ dữ liệu đã được làm sạch.


In [34]:
import pandas as pd
import numpy as np

# Tải dữ liệu đã được làm sạch từ Phần 1
df = pd.read_csv('../Data_main/covid_cleaned_country_data.csv')
df['date'] = pd.to_datetime(df['date']) # Đảm bảo cột date là kiểu datetime


In [19]:
# Lấy dữ liệu mới nhất của mỗi quốc gia
latest_data = df.sort_values('date').groupby('location').last().reset_index()

# Hiển thị 5 dòng đầu để kiểm tra
print(latest_data.head())


         location iso_code continent       date  total_cases  new_cases  \
0     Afghanistan      AFG      Asia 2024-08-04     235214.0        0.0   
1         Albania      ALB    Europe 2024-08-04     335047.0        0.0   
2         Algeria      DZA    Africa 2024-08-04     272139.0       18.0   
3  American Samoa      ASM   Oceania 2024-08-04       8359.0        0.0   
4         Andorra      AND    Europe 2024-08-04      48015.0        0.0   

   new_cases_smoothed  total_deaths  new_deaths  new_deaths_smoothed  ...  \
0               0.000        7998.0         0.0                  0.0  ...   
1               0.000        3605.0         0.0                  0.0  ...   
2               2.571        6881.0         0.0                  0.0  ...   
3               0.000          34.0         0.0                  0.0  ...   
4               0.000         159.0         0.0                  0.0  ...   

   extreme_poverty  cardiovasc_death_rate  diabetes_prevalence  \
0              0.0  

In [36]:
# Kiểm tra các cột có trong DataFrame 'latest_data'
print("Các cột có trong 'latest_data':")
print(latest_data.columns.tolist())

Các cột có trong 'latest_data':
['location', 'iso_code', 'continent', 'date', 'total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths', 'new_deaths', 'new_deaths_smoothed', 'total_cases_per_million', 'new_cases_per_million', 'new_cases_smoothed_per_million', 'total_deaths_per_million', 'new_deaths_per_million', 'new_deaths_smoothed_per_million', 'reproduction_rate', 'total_tests', 'new_tests', 'total_tests_per_thousand', 'new_tests_per_thousand', 'new_tests_smoothed', 'new_tests_smoothed_per_thousand', 'positive_rate', 'tests_per_case', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'new_vaccinations', 'new_vaccinations_smoothed', 'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred', 'new_vaccinations_smoothed_per_million', 'new_people_vaccinated_smoothed', 'new_people_vaccinated_smoothed_per_hundred', 'stringency_index', 'population_density', 'median_age',

In [40]:
# Tính tổng ca nhiễm và tử vong toàn cầu (lấy max từng quốc gia)
global_cases = df.groupby('location')['total_cases'].max().sum()
global_deaths = df.groupby('location')['total_deaths'].max().sum()
global_cfr = (global_deaths / global_cases) * 100

# In kết quả
print("SỐ LIỆU CA NHIỄM & TỬ VONG")
print(f"Tổng ca nhiễm: {int(global_cases):,}")
print(f"Tổng ca tử vong: {int(global_deaths):,}")
print(f"Tỷ lệ tử vong (CFR): {global_cfr:.2f}%")

SỐ LIỆU CA NHIỄM & TỬ VONG
Tổng ca nhiễm: 775,625,912
Tổng ca tử vong: 7,055,673
Tỷ lệ tử vong (CFR): 0.91%


In [42]:
check_df = df.groupby('location')[['people_vaccinated', 'people_fully_vaccinated']].max().reset_index()
check_df = check_df.merge(latest_data[['location', 'population']], on='location')
check_df['vax_rate'] = check_df['people_vaccinated'] / check_df['population'] * 100
print(check_df[check_df['vax_rate'] > 100][['location', 'vax_rate']])

     location    vax_rate
30     Brunei  100.478172
80  Gibraltar  129.066316


In [44]:
# Tính toán số liệu tiêm chủng (lấy max từng quốc gia)
global_pop = latest_data['population'].sum()
global_vaccinated = df.groupby('location')['people_vaccinated'].max().sum()
global_fully_vaccinated = df.groupby('location')['people_fully_vaccinated'].max().sum()

# Tính tỷ lệ
global_vax_rate = (global_vaccinated / global_pop) * 100
global_full_vax_rate = (global_fully_vaccinated / global_pop) * 100

# In kết quả
print("SỐ LIỆU TIÊM CHỦNG")
print(f"Dân số (các quốc gia trong dữ liệu): {int(global_pop):,}")
print(f"Đã tiêm ít nhất 1 liều: {int(global_vaccinated):,}")
print(f"Đã tiêm đủ liều: {int(global_fully_vaccinated):,}")
print(f"Tỷ lệ tiêm chủng (ít nhất 1 liều): {global_vax_rate:.2f}%")
print(f"Tỷ lệ tiêm chủng (đủ liều): {global_full_vax_rate:.2f}%")

SỐ LIỆU TIÊM CHỦNG
Dân số (các quốc gia trong dữ liệu): 5,073,101,103
Đã tiêm ít nhất 1 liều: 5,630,055,208
Đã tiêm đủ liều: 5,176,804,808
Tỷ lệ tiêm chủng (ít nhất 1 liều): 110.98%
Tỷ lệ tiêm chủng (đủ liều): 102.04%


In [45]:
# Sắp xếp và lấy top 10
top_10_cases = latest_data.sort_values(by='total_cases', ascending=False).head(10)


# In kết quả
print("Top 10 quốc gia theo tổng số ca nhiễm:")
print(top_10_cases[['location', 'total_cases']])


Top 10 quốc gia theo tổng số ca nhiễm:
      location  total_cases
42       China   99373219.0
72      France   38997490.0
78     Germany   38437756.0
28      Brazil   37511921.0
105      Japan   33803572.0
11   Australia   11861161.0
8    Argentina   10101218.0
98        Iran    7627863.0
97   Indonesia    6829399.0
43    Colombia    6391876.0


In [24]:
# Tìm giá trị tiêm chủng cao nhất cho mỗi quốc gia

max_vax_indices = df.groupby('location')['people_fully_vaccinated_per_hundred'].idxmax()
max_vax_df = df.loc[max_vax_indices]

# Sắp xếp và lấy top 10
top_10_max_vax = max_vax_df.sort_values(by='people_fully_vaccinated_per_hundred', ascending=False).head(10)

# In kết quả
print("Top 10 quốc gia theo tỷ lệ tiêm chủng đầy đủ cao nhất từng ghi nhận (kèm GDP):")
print(top_10_max_vax[['location', 'people_fully_vaccinated_per_hundred', 'gdp_per_capita']])


Top 10 quốc gia theo tỷ lệ tiêm chủng đầy đủ cao nhất từng ghi nhận (kèm GDP):
              location  people_fully_vaccinated_per_hundred  gdp_per_capita
135410       Gibraltar                               126.89           0.000
51496           Brunei                                99.49       71809.251
157726       Hong Kong                                90.85       56054.920
69761            Chile                                90.29       22767.037
85263             Cuba                                89.67           0.000
71443            China                                89.54       15308.712
65070   Cayman Islands                                88.32       49903.029
58376         Cambodia                                87.63        3645.070
39535           Bhutan                                86.61        8708.597
78143     Cook Islands                                86.47           0.000


In [46]:
# Tìm giá trị total_cases cao nhất cho mỗi quốc gia
max_cases_data = df.groupby('location')['total_cases'].max().reset_index()

# Sắp xếp và lấy top 10
top_10_cases = max_cases_data.sort_values(by='total_cases', ascending=False).head(10)

# In kết quả
print("Top 10 quốc gia theo tổng số ca nhiễm cao nhất (max):")
print(top_10_cases[['location', 'total_cases']])


Top 10 quốc gia theo tổng số ca nhiễm cao nhất (max):
           location  total_cases
224   United States  103436829.0
42            China   99373219.0
96            India   45041748.0
72           France   38997490.0
78          Germany   38437756.0
28           Brazil   37511921.0
198     South Korea   34571873.0
105           Japan   33803572.0
103           Italy   26781078.0
223  United Kingdom   24974629.0


In [48]:
# Tìm giá trị stringency_index cao nhất cho mỗi quốc gia
max_stringency_per_country = df.groupby('location')['stringency_index'].max().reset_index()

# Sắp xếp và lấy top 10
top_10_max_stringency = max_stringency_per_country.sort_values(by='stringency_index', ascending=False).head(10)

# In kết quả
print("Top 10 quốc gia theo chỉ số nghiêm ngặt cao nhất từng ghi nhận:")
print(top_10_max_stringency)


Top 10 quốc gia theo chỉ số nghiêm ngặt cao nhất từng ghi nhận:
               location  stringency_index
118               Libya             100.0
8             Argentina             100.0
62          El Salvador             100.0
58   Dominican Republic             100.0
111              Kuwait             100.0
107              Jordan             100.0
188              Serbia             100.0
203            Suriname             100.0
77              Georgia             100.0
50                 Cuba             100.0


In [50]:
# Tìm giá trị total_deaths cao nhất cho mỗi quốc gia
max_deaths_data = df.groupby('location')['total_deaths'].max().reset_index()

# Sắp xếp và lấy top 10
top_10_deaths = max_deaths_data.sort_values(by='total_deaths', ascending=False).head(10)

# In kết quả
print("Top 10 quốc gia theo tổng số ca tử vong cao nhất (max):")
print(top_10_deaths[['location', 'total_deaths']])



Top 10 quốc gia theo tổng số ca tử vong cao nhất (max):
           location  total_deaths
224   United States     1193165.0
28           Brazil      702116.0
96            India      533623.0
174          Russia      403188.0
134          Mexico      334551.0
223  United Kingdom      232112.0
165            Peru      220975.0
103           Italy      197307.0
78          Germany      174979.0
72           France      168091.0


In [55]:
# Tìm giá trị total_deaths_per_million cao nhất cho mỗi quốc gia
max_deaths_per_million_data = df.groupby('location')['total_deaths_per_million'].max().reset_index()

# Sắp xếp và lấy top 10
top_10_deaths_per_million = max_deaths_per_million_data.sort_values(by='total_deaths_per_million', ascending=False).head(10)

# In kết quả
print("Top 10 quốc gia theo tỷ lệ ca tử vong trên 1 triệu dân cao nhất (max):")
print(top_10_deaths_per_million[['location', 'total_deaths_per_million']])


Top 10 quốc gia theo tỷ lệ ca tử vong trên 1 triệu dân cao nhất (max):
                   location  total_deaths_per_million
165                    Peru                  6601.110
31                 Bulgaria                  5669.613
155         North Macedonia                  5422.131
26   Bosnia and Herzegovina                  5114.820
94                  Hungary                  5065.206
49                  Croatia                  4799.552
194                Slovenia                  4766.858
77                  Georgia                  4519.362
139              Montenegro                  4317.904
53                  Czechia                  4076.466


In [57]:

# Lọc ra các hàng có total_cases > 0 để loại bỏ các ngày không có ca nhiễm
first_case_df = df[df['total_cases'] > 0]

# Tìm ngày đầu tiên có ca nhiễm cho mỗi quốc gia
first_case_dates = first_case_df.groupby('location')['date'].min().reset_index()

# Sắp xếp các quốc gia theo ngày ghi nhận ca đầu tiên và lấy 5 hàng đầu
top_5_first_cases = first_case_dates.sort_values(by='date').head(5)

# In kết quả
print("Top 5 quốc gia ghi nhận ca nhiễm COVID-19 đầu tiên:")
print(top_5_first_cases)


Top 5 quốc gia ghi nhận ca nhiễm COVID-19 đầu tiên:
    location       date
78   Germany 2020-01-05
42     China 2020-01-05
135   Monaco 2020-01-12
104    Japan 2020-01-19
197    Spain 2020-01-19


In [62]:

max_cases_indices = df.groupby('location')['total_cases'].idxmax()
df_max_cases_per_country = df.loc[max_cases_indices]

cases_by_continent_max = df_max_cases_per_country.groupby('continent')['total_cases'].sum().reset_index()

sorted_cases_by_continent = cases_by_continent_max.sort_values(by='total_cases', ascending=False)

# In kết quả
print("Xếp hạng các châu lục theo tổng số ca nhiễm (dựa trên max của mỗi quốc gia):")
print(sorted_cases_by_continent)


Xếp hạng các châu lục theo tổng số ca nhiễm (dựa trên max của mỗi quốc gia):
       continent  total_cases
1           Asia  301532347.0
2         Europe  252642589.0
3  North America  124492666.0
5  South America   68809418.0
4        Oceania   15003352.0
0         Africa   13145540.0


In [63]:
# Tải dữ liệu tổng hợp đã lưu
df_agg = pd.read_csv('../Data_main/covid_owid_aggregated_data.csv')
df_agg['date'] = pd.to_datetime(df_agg['date'])

# Danh sách các mã ISO cần phân tích
income_iso_codes = ['OWID_HIC', 'OWID_LIC', 'OWID_LMC']

# Lọc ra các nhóm thu nhập theo mã ISO
income_groups_df = df_agg[df_agg['iso_code'].isin(income_iso_codes)]

# Hiển thị 5 dòng đầu để kiểm tra
print("Dữ liệu của các nhóm thu nhập:")
print(income_groups_df.head())


Dữ liệu của các nhóm thu nhập:
      iso_code continent               location       date  total_cases  \
9425  OWID_HIC       NaN  High-income countries 2020-12-02          0.0   
9426  OWID_HIC       NaN  High-income countries 2020-12-03          0.0   
9427  OWID_HIC       NaN  High-income countries 2020-12-04          0.0   
9428  OWID_HIC       NaN  High-income countries 2020-12-05          0.0   
9429  OWID_HIC       NaN  High-income countries 2020-12-06          0.0   

      new_cases  new_cases_smoothed  total_deaths  new_deaths  \
9425        0.0                 0.0           0.0         0.0   
9426        0.0                 0.0           0.0         0.0   
9427        0.0                 0.0           0.0         0.0   
9428        0.0                 0.0           0.0         0.0   
9429        0.0                 0.0           0.0         0.0   

      new_deaths_smoothed  ...  extreme_poverty  cardiovasc_death_rate  \
9425                  0.0  ...              0.0      

C:\Users\Admin\AppData\Local\Temp\ipykernel_11612\1031341979.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_agg = pd.read_csv('../Data_main/covid_owid_aggregated_data.csv')


In [64]:
# Tìm các giá trị cao nhất của các cột quan trọng cho mỗi nhóm thu nhập
comparison_max = income_groups_df.groupby('location').agg({
    'total_cases_per_million': 'max',
    'total_deaths_per_million': 'max',
    'people_fully_vaccinated_per_hundred': 'max',
}).reset_index()

# In kết quả so sánh
print("So sánh các chỉ số cao nhất của các nhóm thu nhập:")
print(comparison_max)


So sánh các chỉ số cao nhất của các nhóm thu nhập:
                        location  total_cases_per_million  \
0          High-income countries                351468.50   
1           Low-income countries                  2712.27   
2  Lower-middle-income countries                 28590.73   

   total_deaths_per_million  people_fully_vaccinated_per_hundred  
0                  2455.406                                74.31  
1                    60.721                                27.79  
2                   369.385                                59.82  


In [65]:
# Tìm giá trị total_cases và total_deaths cao nhất cho mỗi nhóm
max_stats_income = income_groups_df.groupby('location').agg({
    'total_cases': 'max',
    'total_deaths': 'max'
})

# Tính toán CFR dựa trên các giá trị max
max_stats_income['case_fatality_rate'] = \
    (max_stats_income['total_deaths'] / max_stats_income['total_cases']) * 100

# In kết quả
print("So sánh Tỷ lệ tử vong (CFR) dựa trên số liệu cao nhất của các nhóm thu nhập:")
print(max_stats_income[['case_fatality_rate']])


So sánh Tỷ lệ tử vong (CFR) dựa trên số liệu cao nhất của các nhóm thu nhập:
                               case_fatality_rate
location                                         
High-income countries                    0.698613
Low-income countries                     2.238761
Lower-middle-income countries            1.291973
